# API to the RSP from NOIRLab's Astro DataLab

**Run this notebook at the NOIRLab Astro DataLab** (https://datalab.noirlab.edu/).

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<br>

**Contact authors:** Melissa Graham and Leanne Guy <br>
**Last verified:** Thu Nov 30 2023 <br>
**Rubin data release:** Data Preview 0.2 (DP0.2) <br>

**Description:**
This tutorial demonstrates how a broker, or a broker user, working at the NOIRLab Astro DataLab (or other platform or personal computer with PyVO installed) can obtain host galaxy information for candidate supernovae identified in the LSST alert stream.

**Credit:** Sections 1 and 2 are based on the <a href="https://dp0-2.lsst.io/data-access-analysis-tools/api-intro.html">Introduction to the RSP API Aspect</a> webpage which had major contributions from Douglas Tucker.

**Requirements:** Accounts in the NOIRLab Astro DataLab and the Rubin Science Platform (RSP) at https://data.lsst.cloud/. Only individuals with <a href="https://docushare.lsst.org/docushare/dsweb/Get/RDO-013">Rubin data rights</a> may have an RSP account. See the <a href="https://dp0-2.lsst.io/dp0-delegate-resources/index.html#delegate-homepage-getting-started-checklist">getting started with DP0.2 checklist</a> for instructions about how to request an RSP account.

## 1. Set up RSP token in DataLab

Start a JupyterLab session at NOIRLab's Astro DataLab (https://datalab.noirlab.edu/).

The instructions in Section 1 only need to be done once in the DataLab 
(and they also apply to personal computers or other JupyterLab platforms).

After completing Section 1 once, any other notebook run in the DataLab
can start with the imports and TAP credential code cells in Section 2.

### 1.1. Generate a new RSP token

**THE TOKEN IS A PASSWORD.**
Keep it secret. Keep it safe.

**NEVER DISPLAY THE TOKEN.**
Do not even `print(token)` from a code cell. Avoid accidental sharing.

**NEVER SAVE THE TOKEN IN A GIT-TRACKED FILE.**
Do not let the internet see the token.


Follow steps 1 through 5 of <a href="https://nb.lsst.io/environment/tokens.html#using-a-token-outside-the-science-platform">these instructions to obtain a token for an RSP account</a>, which are summarized below.
 * go to data.lsst.cloud
 * use the drop-down menu at upper right to log in
 * select "Security Tokens" from the upper-right menu
 * on the new page, under "User Tokens" click "Create Token"
 * in the pop-up panel, enter a name, e.g., "noirlab-astro-datalab"
 * click box to select only "read:tap"
 * under "Expires" select "Never"
 * click "Create"
 * the token will be a long string of characters
 * in the new pop-up panel, copy the token to a safe file

Proceed to Section 1.2.

### 1.2. Create RSP token file in DataLab

Create a hidden file `~/.rsp-tap.token` containing only the token.
set chmod 600

**Unfamiliar with editing hidden files (dot-files)?**<br>
Here is a step-by-step process.
 * use the left sidebar to navigate to the home directory
   * to do this, click the folder icon immediately under the search bar at left
   * if there is no left sidebar, go "View" --> "Show Left Sidebar"
 * at upper left, click the launcher button (blue square with a + sign)
 * in the new tab which opens, click "Text File" in the bottom row
 * paste the token into the newly opened text file
 * at upper left click "File" then "Save As" and name the file "temp.txt"
 * the file will be visible in the home directory in the left sidebar
 * again click the launcher button, then select terminal
 * in the terminal:
   * type `cd ~` and hit the return (or enter) key to navigate to the home directory
   * type `mv temp.txt .rsp-tap.token` and hit return to rename the file
   * type `chmod 600 .rsp-tap.token` and hit return to give read/write permission to user only
   * type `ls -lah .rsp-tap.token` and hit return to view the file's <a href="https://en.wikipedia.org/wiki/File-system_permissions#Notation_of_traditional_Unix_permissions">Unix permissions</a>
   * confirm the permissions look like: `-rw-------`
   * type `more ~/.rsp-tap.token` and hit return to view the token
 * close the terminal and text edit tabs, they are no longer needed

## 2. Set up RSP TAP service

Import <a href="https://pyvo.readthedocs.io/en/latest/">PyVO</a>, getpass, pandas,
numpy, and matplotlib.pyplot.

In [1]:
import pyvo
import getpass
import pandas
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

Get the name of the token file.

In [2]:
my_username = getpass.getuser()
token_filename = '/dlusers/'+my_username+'/.rsp-tap.token'

Option to print the username and token file name.

In [ ]:
# print(my_username)
# print(token_filename)

Get the token from the token file, and assert that it is not `None`.

If the following cell returns a message, then the `token` is `None` and there might be something wrong with the token file.
No message means all is OK.

In [3]:
with open(token_filename, 'r') as f:
    token = f.readline()
assert token is not None

**Do not** `print(token)`. The risk of displaying the token, 
then saving and sharing (or git-tracking) this file, is avoided
if the token is never displayed in the first place.

Establish the RSP TAP service (`rsp_tap`) using pyvo. 

The TAP user (`x-oauth-basic`), 
security method (`ivo://ivoa.net/sso#BasicAA`),
and the DP0.2 TAP endpoint (`https://data.lsst.cloud/api/tap`)
will be the same for everyone.
Note that the token is passed in the second line.

Assert that the `rsp_tap` is not `None`, and that the `rsp_tap.baseurl` matches the desired TAP endpoint. 
If there is no output message, it means all is OK.

In [4]:
cred = pyvo.auth.CredentialStore()
cred.set_password("x-oauth-basic", token)
credential = cred.get("ivo://ivoa.net/sso#BasicAA")
rsp_tap_url = 'https://data.lsst.cloud/api/tap'
rsp_tap = pyvo.dal.TAPService(rsp_tap_url, credential)
assert rsp_tap is not None
assert rsp_tap.baseurl == rsp_tap_url

### 2.1. Optional DP0.2 test query

Option to do a simple query to retrieve TAP schemas available in the Rubin Science Platform (data.lsst.cloud),
display the results, and then delete them.

In [ ]:
# query = "SELECT * FROM tap_schema.schemas"
# results = rsp_tap.run_sync(query).to_table()

In [ ]:
# results

In [ ]:
# del results

### 2.2. Optional test of DP0.3 TAP endpoint

Solar System objects (SSO) simulated as part of Rubin Observatory's Data Preview 0.3 (DP0.3) are available
at a unique TAP URL. 

Option to test the connection and a simple query to the DP0.3 data products.

In [ ]:
# rsp_tap_url_sso = 'https://data.lsst.cloud/api/ssotap'
# rsp_tap_sso = pyvo.dal.TAPService(rsp_tap_url_sso, credential)
# assert rsp_tap_sso is not None
# assert rsp_tap_sso.baseurl == rsp_tap_url_sso

In [ ]:
# query_sso = "SELECT * FROM tap_schema.schemas"
# results_sso = rsp_tap_sso.run_sync(query_sso).to_table()

In [ ]:
# results_sso

In [ ]:
# del results_sso, rsp_tap_url_sso, rsp_tap_sso, query_sso

## 3. Obtain potential host galaxy information

**The scientific scenario for this exercise** is that the date is 60965 MJD, 
and an alert broker has identified five
`DiaObjects` of interest from the LSST alert stream.
These `DiaObjects` have rising light curves with at least 5 detections
in the past 20 days and have reached a brightness < 22 mag.
They are potential Type Ia supernovae, and might be suitable for 
a hypothetical follow-up program with Gemini.
However, more contextual information is needed.

**The LSST alert packets will contain** the `objectId` values for
the three nearest stars and three nearest galaxies in the most recent 
LSST annual data release, but no further data about the objects themselves.
LSST alert packets will also contain the three nearest extended objects using separation
distances calculated from the second moments of the object's luminosity profile,
and the neareset low-redshift galaxy from a pre-established catalog.

**In this demo, only the three nearest stars and galaxies based on 2D sky separations are simulated.**
For DP0.2, the `DiaObject` table does not include these columns.
For the purpose of this demo, the `refExtendedness` parameter in the `Object` catalog was used 
to identify the three nearest "stars" (extendedness = 0) and "galaxies" (extendedness = 1),
and identify the `Objects` that populate the dataframe in Section 3.1.
For a full description of the nearby-object data that will be in the LSST alert packets,
see Table 3 of the Rubin <a href="https://lse-163.lsst.io/">Data Products Definitions Document</a> (DPDD)
and the Rubin Data Management Tech Note <a href="https://dmtn-151.lsst.io/">Host Galaxy Association for DIAObjects</a> (DMTN-151).

**The task at hand** is to use the `objectId` to retrieve data
for the nearby stars and galaxies from the latest annual LSST Data
Release (in this case, the DP0.2 `Object` table)
and use that data to prioritize the `DiaObjects` for follow-up.

### 3.1. Simulate nearby-object data from LSST alert packets

The following cell creates a pandas dataframe that mimics some of the 
nearby-object data that a broker filter would have from the LSST alert packets,
for the scenario described above.

The dataframe contains a list of five `DiaObjects` of interest,
their coordinates,
and the three nearest stars and galaxies for each.

 * `diaObjectId` : identifier in the DP0.2 `DiaObject` table for the candidate supernova
 * `diaObject_coord` : coordinates [RA, Dec] in decimal degrees for the candidate supernova
 * `stars_objId` : identifier in the DP0.2 `Object` table for the three nearest stars
 * `stars_2Ddist` : the 2D sky distance in arcseconds between `DiaObject` and nearby star
 * `gals_objId` : identifier in the DP0.2 `Object` table for the three nearest galaxies
 * `gals_2Ddist` : the 2D sky distance in arcseconds between `DiaObject` and nearby galaxy's center
 
In the scientific scenario of this exercise, these are five potential Type Ia supernovae.

In [5]:
d = {'diaObjectId' : [1568026726510894110, 1569909090417642499, 1653700672547196623, 
                      1734140943235288573, 1825796232526695593],
     'diaObject_coord' : [[63.6025914, -38.634654],
                          [69.9257038, -38.1424959],
                          [70.8210894, -35.9915118],
                          [52.5432991, -34.9028848],
                          [71.7356252, -34.2191764]],
     'stars_objId' : [[1568026726510919263, 1568026726510919265, 1568026726510919393],
                      [1569425305301455018, 1569425305301455016, 1569425305301455020],
                      [1653700672547231401, 1653700672547231164, 1653700672547231611],
                      [1734140943235326652, 1734140943235326653, 1734140943235298415],
                      [1739084347513803590, 1739084347513777036, 1739084347513805129]],
     'stars_2Ddist' : [[0.03, 4.45, 8.53],
                       [6.47, 8.15, 8.62],
                       [9.94, 11.98, 12.03],
                       [0.02, 2.95, 9.66],
                       [9.48, 10.67, 12.13]],
     'gals_objId' : [[1568026726510919266, 1568026726510919261, 1568026726510919497],
                     [1569425305301455007, 1569425305301455003, 1569425305301455014],
                     [1653700672547231391, 1653700672547231402, 1653700672547231397],
                     [1734140943235326493, 1734140943235293084, 1734140943235326492],
                     [1739084347513803559, 1739084347513803574, 1739084347513803571]],
     'gals_2Ddist' : [[3.15, 4.62, 5.08],
                      [0.02, 3.08, 3.98],
                      [2.13, 2.58, 4.34],
                      [4.7, 5.98, 6.11],
                      [0.03, 4.64, 5.93]]}

df = pandas.DataFrame(data=d)
del d

Option to display the dataframe.

In [ ]:
# df

### 3.2. Retrieve object data from the RSP's DP0.2 catalog

Choose to explore the potential host galaxies for the first of the five `DiaObjects`.

Set the value of `diao_index` to 0.

In [ ]:
diao_index = 0

Create `list_objId`, a string containing a comma-separated list of the three `objectId` for
the three nearest galaxies to the selected `DiaObject`.

In [ ]:
temp = np.asarray(df['gals_objId'][diao_index], dtype='int')
list_objId = "(" + ','.join(['%20i' % num for num in temp]) + ")"
del temp
print(list_objId)

Create a query to retreive object astrometry, shape, size, and photometry measurements from the DP0.2 `Object` catalog.

See the <a href="https://dp0-2.lsst.io/data-products-dp0-2/index.html#dp0-2-data-products-definition-document-dpdd">DP0.2 DPDD</a> and <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html">DP0.2 schema browser</a> for more information about the columns.

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, refExtendedness, "\
        "shape_xx, shape_xy, shape_yy, "\
        "scisql_nanojanskyToAbMag(g_cModelFlux) AS g_cModelMag, "\
        "scisql_nanojanskyToAbMag(r_cModelFlux) AS r_cModelMag, "\
        "scisql_nanojanskyToAbMag(i_cModelFlux) AS i_cModelMag "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE objectId IN "+list_objId
del list_objId

Execute the query using the `rsp_tap` service, and store the results in `galaxies` as a table.

> **Keep in mind that this query is *remote*! 
It is NOT retrieving data from NOIRLab's archive (which is co-located with the Astro DataLab) but from the Rubin Observatory's Data Preview 0.2 dataset via the Rubin Science Platform, which is deployed in the Google Cloud.**

In [ ]:
galaxies = rsp_tap.search(query).to_table()

Option to view the remotely retrieved data table.

In [ ]:
# galaxies

### 3.3. Calculate additional galaxy properties

#### Galaxy colors

The `Objects` table for DP0.2 does not have a photometric redshift,
but see Rubin Data Management Tech Note 
<a href="https://dmtn-049.lsst.io/">A Roadmap to Photometric Redshifts for the LSST Object Catalog</a>
(DMTN-049) for future plans.

Instead, calculate the colors of the galaxies in $g-r$ and $r-i$ magnitude, and add them to the `galaxies` table.

In [ ]:
galaxies['gr_clr'] = galaxies['g_cModelMag'] - galaxies['r_cModelMag']
galaxies['ri_clr'] = galaxies['r_cModelMag'] - galaxies['i_cModelMag']

In [ ]:
# galaxies

#### Separation in elliptical radii

The 2D sky separation is not as good a host indicator as, for example, the separation distance in elliptical radii
that is based on the second moments of the galaxy's luminosity profile.

In the future, the LSST alert packet will contain this separation distance for the three nearest extended objects.

For this demo, calculate both the 2D sky separation and the elliptical radius separation for each galaxy,
and add them to the `galaxies` table.

In [ ]:
galaxies['ell_rad'] = np.zeros(3, dtype='float')
galaxies['2Ddist'] = np.zeros(3, dtype='float')

snra = df['diaObject_coord'][diao_index][0]
sndec = df['diaObject_coord'][diao_index][1]
sncoord = SkyCoord(snra, sndec, unit='deg')

for i in range(3):
    objra = galaxies['coord_ra'][i]
    objdec = galaxies['coord_dec'][i]
    objcoord = SkyCoord(objra, objdec, unit='deg')
    del objra, objdec
    
    temp = objcoord.separation(sncoord)
    galaxies['2Ddist'][i] = temp.arcsec
    del temp
    
    temp = objcoord.spherical_offsets_to(sncoord)
    xr = 3600.0 * temp[0].deg
    yr = 3600.0 * temp[1].deg
    del temp, objcoord
    
    Ixx = galaxies['shape_xx'][i]
    Iyy = galaxies['shape_yy'][i]
    Ixy = galaxies['shape_xy'][i]
    Cxx = Iyy / ((Ixx * Iyy) - Ixy)
    Cyy = Ixx / ((Ixx * Iyy) - Ixy)
    Cxy = -2.0 * (Ixy) / ((Ixx * Iyy) - Ixy)
    galaxies['ell_rad'][i] = np.sqrt((Cxx * xr**2) + (Cyy * yr**2) + (Cxy * xr * yr))

    del Ixx, Iyy, Ixy, Cxx, Cyy, Cxy

del snra, sndec, sncoord

In [ ]:
galaxies

### 3.4. Interpret derived data for nearby galaxies

For the `DiaObject` with `diaObjectId` = 1568026726510894110 (`diao_index` = 0),
interpret the new information about nearby galaxies that was obtained from the RSP.

For the nearest galaxy by 2D sky separation, print the `objectId`, separations, and colors.

In [ ]:
mx = np.argmin(galaxies['2Ddist'])
print(galaxies['objectId'][mx], 
      galaxies['2Ddist'][mx], galaxies['ell_rad'][mx],
      galaxies['gr_clr'][mx], galaxies['ri_clr'][mx])
del mx

For the nearest galaxy by elliptical radii separation, print the `objectId`, separations, and colors.

In [ ]:
mx = np.argmin(galaxies['ell_rad'])
print(galaxies['objectId'][mx], 
      galaxies['2Ddist'][mx], galaxies['ell_rad'][mx],
      galaxies['gr_clr'][mx], galaxies['ri_clr'][mx])
del mx

**Summary:** The galaxy that is nearest by 2D sky separation (3.15") is not the best candidate host galaxy:
the best candidate is the one with a larger 2D sky separation (4.62") but a smaller offset in 
terms of elliptical radii ($R=1.02$), which takes into account the size of the galaxy.
Furthermore, the best candidate also has redder colors, ($g-r$ and $r-i>0$), which is more typical 
for the host galaxies of Type Ia supernovae.

In [ ]:
del diao_index, galaxies

### 3.5. Exercises for the learner

 1. Repeat the analysis in Section 3 by choosing a different `DiaObject` to explore.
 2. The three nearest stars are also provided, and some have low 2D sky separations. What's up with them?
 3. Explore the <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html">DP0.2 schema browser</a> and retrieve additional columns.

## 4. Retrieve and display a light curve

For the science case of a new supernova the LSST alert packet
would contain the difference-image light curves, and the following
example of retrieving photometry from the LSST annual data release
(in this case, DP0.2) would be uncecessary.

But as a demo, retrieve the difference-image 5-sigma detections
for one of the `DiaObjects` up to the date of MJD 60965,
and plot the light curve.

### 4.1. Set light curve plot parameters

These color and symbol combinations are colorblind-friendly.

In [ ]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

### 4.2. Retrieve the light curve for one `DiaObject`

Recall that in this scientific scenario of Section 3, the MJD is 60965.

**Why use midPointTai < 60965?**
The DP0.2 tables contain difference-image detections for all five years,
but in the query below only those up to the night of 60965 are retreived.
This mimics what a broker user would be working with, when deciding
whether or not to follow-up these `DiaObjects` on MJD 60965,
as described in the science scenario at the start of Section 3.

**Why use scisql_nanojanskyToAbMag?**
It is safe in this situation only because the photometry for
5-sigma difference image detections of a rising supernova are
being returned, and the difference-image flux is not expected 
to be negative in this situation.
However, generally, it is not advised to use `scisql_nanojanskyToAbMag`
with difference-image fluxes.

Descriptions and units for the columns of the `DiaSource` catalog are
available in the <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html">DP0.2 schema browser</a>.

In [ ]:
diao_index = 0
temp_string = str(df['diaObjectId'][diao_index])
diasources = rsp_tap.search("SELECT midPointTai, filterName, "
                            "scisql_nanojanskyToAbMag(psFlux) AS psAbMag "
                            "FROM dp02_dc2_catalogs.DiaSource  "
                            "WHERE diaObjectId = "+temp_string+" "
                            "AND midPointTai < 60965").to_table()
del diao_index, temp_string

Option to display the data retrieved from the Rubin Science Platform.

In [ ]:
# diasources

### 4.3. Display the light curve

Plot the light curve with matplotlib. 

In [ ]:
fig = plt.figure(figsize=(6, 4))

for f, filt in enumerate(plot_filter_labels):
    fx = np.where(diasources['filterName'][:] == filt)[0]
    if len(fx) > 0:
        plt.plot(diasources['midPointTai'][fx]-60965, 
                 diasources['psAbMag'][fx],
                 plot_filter_symbols[filt], 
                 ms=10, mew=0, alpha=0.5,
                 color=plot_filter_colors[filt],
                 label=plot_filter_labels[f])
    del fx

plt.xlabel('Days Ago (MJD-60965)')
plt.ylabel('Apparent AB Magnitude')
plt.gca().invert_yaxis()
plt.title('5-Sigma Difference-Image Detections')
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
del diasources

### 4.4. Exercises for the learner

 1. Repeat Section 4 but select a different `DiaObject` and display its light curve.
 2. Remove the limit of MJD<60965 from the query and plot the full light curve.
 3. Obtain the forced-photometry light curve from the `ForcedSourceOnDiaObject` catalog (do not use `scisql_nanojanskyToAbMag`) and plot it instead.

## 5. Retrieve and display a large, deep cutout image

For the science case of a new supernova the LSST alert packet
would contain the difference-image and the reference-image stamps,
but they are relatively small cutouts: no smaller than 30 x 30 pixels
(6" x 6").
A larger cutout of a the deeply coadded image would be
useful for interpreting the host galaxy situation, or
even creating finder charts for spectroscopic observations.

Select the first `DiaObject` on the list, and obtain its coordinates.

In [6]:
diao_index = 0
snra = df['diaObject_coord'][diao_index][0]
sndec = df['diaObject_coord'][diao_index][1]

### 5.1. Query for the i-band deepCoadd image

Create the query for the i-band deeply coadded image (`deepCoadd_calexp`) that overlaps the `DiaObject` coordinates.

It is recommended to always select all rows with `*` when querying the `ivoa.ObsCore`.

In [7]:
query = "SELECT * FROM ivoa.ObsCore "\
        "WHERE dataproduct_type = 'image' "\
        "AND obs_collection = 'LSST.DP02' "\
        "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' "\
        "AND CONTAINS(POINT('ICRS', "+str(snra)+", "+str(sndec)+"), s_region) = 1 "\
        "AND lsst_band = 'i' "
print(query)

SELECT * FROM ivoa.ObsCore WHERE dataproduct_type = 'image' AND obs_collection = 'LSST.DP02' AND dataproduct_subtype = 'lsst.deepCoadd_calexp' AND CONTAINS(POINT('ICRS', 63.6025914, -38.634654), s_region) = 1 AND lsst_band = 'i' 


Query the `ivoa.ObsCore`, which is the DP0.2 images available via the Rubin Science Platform deployed in the Google Cloud.

Assert that the `results` table contains only one row, there will be only one overlapping i-band deep coadd image.

In [8]:
results = rsp_tap.search(query)
assert len(results) == 1

Show the results in a table.

In [9]:
results.to_table().show_in_notebook()

idx,access_format,access_url,calib_level,dataproduct_subtype,dataproduct_type,em_max,em_min,em_res_power,em_xel,facility_name,instrument_name,lsst_band,lsst_ccdvisitid,lsst_detector,lsst_filter,lsst_patch,lsst_tract,lsst_visit,o_ucd,obs_collection,obs_id,obs_publisher_did,obs_title,pol_xel,s_dec,s_fov,s_ra,s_region,s_resolution,s_xel1,s_xel2,t_exptime,t_max,t_min,t_resolution,t_xel,target_name
,,,,,,m,m,,,,,,,,,,,,,,,,,,deg,deg,deg,,arcsec,,,s,d,d,s,,
0,application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/54245023-b11a-4a69-b1ca-f2ef55527547,3,lsst.deepCoadd_calexp,image,8.18e-07,6.91e-07,--,--,Rubin-LSST,,i,--,--,,2,3638,--,phot.count,LSST.DP02,DC2-3638-2,,deepCoadd_calexp - i - DC2-3638-2,--,-38.59736835686038,0.32605043687272384,63.703967530995406,POLYGON ICRS 63.554916 -38.711509 63.853909 -38.710784 63.852553 -38.483043 63.554494 -38.483762,--,--,--,--,--,--,--,--,


The `access_url` column contains the URL for the retrievable image, 
and the `access_format` specifies the format (content type) of the data product.

There are two ways to obtain these from the table.

In [ ]:
# print(results[0].getdataurl())
# print(results[0].getdataformat())

In [ ]:
# print(results[0]['access_url'])
# print(results[0]['access_format'])

The `access_format` indicates that the `access_url` is a <a href="https://www.ivoa.net/documents/DataLink/">DataLink</a> service.
DataLink is an IVOA data access protocol that provides a linking mechanism to metadata about a dataset, and the dataset itself.

### 5.2. Use DataLink to retrieve the image 



In [10]:
from pyvo.dal.adhoc import DatalinkResults

In [11]:
auth_session = rsp_tap._session
print(auth_session)

In [15]:
dl_results = DatalinkResults.from_result_url(results[0]['access_url'], session=auth_session)

E19: None:1:0: E19: File does not appear to be a VOTABLE